In [1]:
# dependencies
import pandas as pd
import os
from datetime import datetime
from geopy import distance

In [2]:
# years to iterate
search_year = ["2019","2020","2021"]

In [3]:
# get the monthly file names for all of the years combines 
files = []
for year in search_year:
    month_files = os.listdir('monthly_data/' + year + '_monthly_files')
    month_files.sort()
    files.extend([[year,month_files]])

# Demographics Data for 2019 - July 2021
## Files are too large for Tableau
1. create a function to return demographic data from the larger sets
2. function iterates through each month in the selected years
3. function yields a limited number of columns
4. function concatenates the limited data into one df
6. export to csv
7. load into Tableau as a manageable filesize

In [4]:
# date formats changed in 2021, so there are two data parsers
def new_date_parser(x): return datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
def old_date_parser(x): return datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")


# columns changed in 2021, these are the new columns - use to rename old columns with new names
map_old_cols = [
    'trip_duration',
    'started_at',
    'ended_at',
    'start_station_id',
    'start_station_name',
    'start_lat',
    'start_lng',
    'end_station_id',
    'end_station_name',
    'end_lat',
    'end_lng',
    'bike_id',
    'member_casual',
    'birth_year',
    'gender'
]

# generator function to import data from csv files and union all files


def load_files(files):
    for col in files:
        for row in col[1]:
            # data types and column names changed in Feb 2021
            if row < "202102":
                yield(
                    pd.read_csv('monthly_data/' + col[0] + '_monthly_files/' + row,
                                names=map_old_cols,
                                header=0,
                                index_col=False,
                                dtype={'start_station_id': str,
                                       'end_station_id': str},
                                usecols=[
                        'started_at',
                        'ended_at',
                        'start_station_id',
                        'end_station_id',
                        'member_casual',
                        'trip_duration',
                        'birth_year',
                        'gender'
                    ],
                        parse_dates=['started_at', 'ended_at'],
                        date_parser=old_date_parser
                    ).replace({'Subscriber': 'member', 'Customer': 'casual'})[
                        [
                            'started_at',
                            'ended_at',
                            'start_station_id',
                            'end_station_id',
                            'member_casual',
                            'trip_duration',
                            'birth_year',
                            'gender'
                        ]
                    ]
                )

            else:
                yield(
                    pd.read_csv('monthly_data/' + col[0] + '_monthly_files/' + row,
                        dtype={'start_station_id': str, 'end_station_id': str},
                        usecols=['started_at', 'ended_at', 'start_station_id', 'end_station_id', 'member_casual'],
                        parse_dates=['started_at', 'ended_at']).assign(trip_duration=lambda x: x.ended_at - x.started_at).astype({'trip_duration': 'timedelta64[s]'})
                )

# combine the limited data sets into one file
demographic_data = pd.concat(load_files(files))


In [5]:
demographic_data

,started_at,ended_at,start_station_id,end_station_id,member_casual,trip_duration,birth_year,gender
0,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160,3283,member,320.0,1971.0,1.0
1,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519,518,member,316.0,1964.0,1.0
2,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171,3154,member,591.0,1987.0,1.0
3,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504,3709,member,2719.0,1990.0,1.0
4,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229,503,member,303.0,1979.0,1.0
...,...,...,...,...,...,...,...,...
3084532,2021-07-08 14:08:12.000,2021-07-08 14:35:15.000,4546.04,6022.04,casual,1623.0,NaN,NaN
3084533,2021-07-24 12:26:49.000,2021-07-24 12:47:18.000,6190.08,5184.08,casual,1229.0,NaN,NaN
3084534,2021-07-04 17:11:16.000,2021-07-04 17:18:23.000,4546.04,4611.03,casual,427.0,NaN,NaN
3084535,2021-07-30 09:43:46.000,2021-07-30 09:57:50.000,6190.08,5484.09,member,844.0,NaN,NaN


In [6]:
# confirming data returns what I need
membership_status = demographic_data.groupby([demographic_data.started_at.dt.date, demographic_data['start_station_id'], demographic_data['member_casual']])['start_station_id'].count().reset_index(name="departures")

In [7]:
membership_status

,started_at,start_station_id,member_casual,departures
0,2019-01-01,119,casual,4
1,2019-01-01,119,member,1
2,2019-01-01,120,casual,4
3,2019-01-01,120,member,15
4,2019-01-01,127,casual,9
...,...,...,...,...
1712406,2021-07-31,8795.03,member,1
1712407,2021-07-31,8799.01,casual,5
1712408,2021-07-31,8811.01,casual,3
1712409,2021-07-31,8841.03,casual,8


In [8]:
# confirming data returns what I need
gender_ridership = demographic_data.groupby([demographic_data.started_at.dt.date, demographic_data['start_station_id'], demographic_data['gender']])['start_station_id'].count().reset_index(name="departures")

In [9]:
gender_ridership


,started_at,start_station_id,gender,departures
0,2019-01-01,119,0.0,4
1,2019-01-01,119,1.0,1
2,2019-01-01,120,0.0,3
3,2019-01-01,120,1.0,11
4,2019-01-01,120,2.0,5
...,...,...,...,...
1845683,2021-01-31,82,1.0,12
1845684,2021-01-31,82,2.0,4
1845685,2021-01-31,83,0.0,1
1845686,2021-01-31,83,1.0,17


In [43]:
# create new columns for month and year to use in Tableau dashboard
demographic_data['ride_month'] = demographic_data['started_at'].dt.month
demographic_data['ride_year'] = demographic_data["started_at"].dt.year


In [44]:
# confirming results
age_ridership = demographic_data.groupby([demographic_data['birth_year'], demographic_data['start_station_id'], demographic_data['ride_year'], demographic_data['ride_month']]).agg(departures=('member_casual', 'count'), avg_trip_duration =('trip_duration', 'mean')).reset_index()

age_ridership

,birth_year,start_station_id,ride_year,ride_month,departures,avg_trip_duration
0,1857.0,146,2019,3,1,1055.0
1,1857.0,265,2019,3,1,1774.0
2,1857.0,3737,2019,3,1,2488.0
3,1857.0,473,2019,3,1,1825.0
4,1857.0,531,2019,3,2,235.0
...,...,...,...,...,...,...
1139195,2004.0,83,2020,11,1,548.0
1139196,2004.0,83,2021,1,1,375.0
1139197,2005.0,3158,2021,1,1,1068.0
1139198,2005.0,3172,2021,1,1,1652.0


In [47]:
# add column for distance between starting and ending station
demographic_data['miles'] = distance.distance(demographic_data['']).miles

538.3904453677204


In [ ]:
# write data to csv
demographic_data.to_csv('annual_data/demographic_data.csv', index = False)